In [ ]:
!pip install simpletransformers
!pip install spacy==3.2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 4.0 MB/s 
     |████████████████████████████████| 362 kB 26.2 MB/s 
     |████████████████████████████████| 1.2 MB 43.5 MB/s 
     |████████████████████████████████| 1.8 MB 9.6 MB/s 
     |████████████████████████████████| 4.4 MB 34.8 MB/s 
     |████████████████████████████████| 43 kB 1.3 MB/s 
     |████████████████████████████████| 6.6 MB 9.8 MB/s 
     |████████████████████████████████| 9.1 MB 27.1 MB/s 
     |████████████████████████████████| 101 kB 9.1 MB/s 
     |████████████████████████████████| 596 kB 42.6 MB/s 
     |████████████████████████████████| 181 kB 23.0 MB/s 
     |████████████████████████████████| 145 kB 37.4 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 140 kB 52.3 MB/s 
     |████████████████████████████████| 1.1 MB 37.5 MB/s 
     |████████████████████████████████|

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.0 MB 3.7 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.1
    Uninstalling spacy-3.3.1:
      Successfully uninstalled spacy-3.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rich 12.4.4 requires typing-extensions<5.0,>=4.0.0; python_version < "3.9", but you have typing-extensions 3.10.0.2 which is incompatible.
en-core-web-sm 3.3.0 requires spacy<3.4.0,>=3.3.0.dev0, but you have spacy 3.2.4 which is incompatible.


In [ ]:
import requests
from tqdm import tqdm
from zipfile import ZipFile
import os 

import shutil

DATASET_NAME = 'obras'
BASE_DIR = DATASET_NAME

src = '/content/drive/MyDrive/punct-dts.zip'
shutil.copyfile(src, 'punct-dts.zip')
shutil.unpack_archive('punct-dts.zip')
shutil.unpack_archive(f'punct-{BASE_DIR}.zip')


In [ ]:
from collections import Counter
import pandas as pd
import re 
from collections import defaultdict
from spacy.matcher import Matcher
from spacy.tokens import Span
from tqdm import tqdm
import spacy

In [ ]:
dataset = {filename.replace('.csv', ''): pd.read_csv(os.path.join(BASE_DIR, filename)).dropna() for filename in
           os.listdir(BASE_DIR)}

In [ ]:
nlp = spacy.blank('pt')

In [ ]:
def preprocess_data(dataframe):
     
    TOTAL = len(list(dataframe.groupby("sentence_id")))
    data = []
    for _, group in tqdm(dataframe.groupby("sentence_id"), total=TOTAL):
        text = " ".join(group.words)
        doc = nlp.make_doc(text)
        ents = []
        count_dict = defaultdict(lambda : 0)
        for i, label in enumerate(group.labels):
            if label != "O":
                matching_word = group.words.tolist()[i]
                span = Span(doc, i, i+1, label=label)
                ents.append((span.start_char, span.end_char, label.replace('I-','')))
                count_dict[matching_word] += 1
        ent = (text, {
            'entities': ents
        })
        data.append(ent)
    return data

In [ ]:
dataset['test'].dropna()

,sentence_id,labels,words
0,42,O,a
1,42,I-COMMA,morte
2,42,O,ainda
3,42,O,que
4,42,O,seja
...,...,...,...
31994,254776,O,deu
31995,254776,O,lhe
31996,254776,O,o
31997,254776,O,prêmio


In [ ]:
BASE_MODEL_DIR = './drive/Shareddrives/MEC - Correção textual/PLN/Notebooks/Pontuação/models'
MODEL_NAME = 'portuguese-tedtalk2012'
zipfile = ZipFile(os.path.join(BASE_MODEL_DIR, f'{MODEL_NAME}.zip'))
zipfile.extractall(path=MODEL_NAME)

In [ ]:
from simpletransformers.ner import NERModel, NERArgs
import torch

model_args = NERArgs()
model_args.labels_list = ["O", "COMMA", "PERIOD", "QUESTION"]

model = NERModel(
    "bert",
    f"./{MODEL_NAME}/",
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

In [ ]:


info = {
      'sentences':df.sentence_id.unique().shape[0],
      'words':df.words.shape[0],
      'labels': {
    'O': df[df['labels']=='O'].shape[0],
    'PERIOD':df[df['labels']=='I-PERIOD'].shape[0],
    'COMMA':df[df['labels']=='I-COMMA'].shape[0],
    'QUESTION':df[df['labels']=='I-QUESTION'].shape[0]
    }
  }
  
pd.DataFrame(info).to_csv() 


In [ ]:
DATA = preprocess_data(df)

In [ ]:
y_true = []
texts = []
for _, group in df.groupby("sentence_id"):
        text = " ".join(group.words)
        texts.append(text)
        y_true.append(group.labels.apply(lambda label: label.replace("I-","") ).tolist())

In [ ]:
len(y_true)

In [ ]:
texts[:5]

In [ ]:
predictions = model.predict(texts)

In [ ]:
y_pred = []
for i,pred in enumerate(predictions[0], 1):
  y_pred.append(list(map(lambda item:list(item.values())[0], pred)))

In [ ]:
len(y_pred[0])

In [ ]:
from spacy.tokens import Span
from spacy.tokens import Doc
from spacy.training import Example

In [ ]:
examples = []

predictions_ner = predictions[0]
for k, (text, entities) in enumerate(DATA):
   
    doc = nlp(text)
    doc.set_ents([Span(doc, i, i+1, item) 
               for i, item in enumerate(y_pred[k]) 
                if item !="O"])
  
    example = Example.from_dict(doc, entities)
    examples.append(example)

In [ ]:
from spacy.scorer import Scorer, get_ner_prf 
#Compute micro-PRF and per-entity PRF scores.
scores = get_ner_prf(examples)
micro_avg = {
    'f':scores.pop('ents_f'),
    'p':scores.pop('ents_p'),
    'r':scores.pop('ents_r')
} 
scores.update(scores.pop('ents_per_type'))
scores.update({'micro_avg':micro_avg})
scores

In [ ]:
DATASET_NAME

In [ ]:
RESULTS_DIR = './drive/Shareddrives/MEC - Correção textual/PLN/Notebooks/Pontuação/Resultados'
path_to_result = os.path.join(RESULTS_DIR,f'{DATASET_NAME}.xlsx')
if os.path.exists(path_to_result):
  mode = 'a'
else:
  mode = 'w'
mode

In [ ]:
sheet_name = 'tts-portuguese-corpus'
sheet_name

In [ ]:

with pd.ExcelWriter(path_to_result, mode=mode) as writer:  
      pd.DataFrame(scores).T.to_excel(writer, sheet_name=sheet_name)